# 目標

## 機械学習の基礎を理解して、独自で基本的な学習モデルのコードを実装できるようになる。

# 学習内容

## - AIとは何か
## (- ベイジアンフィルターの実装?)
## - 機械学習の基本原理とは
## - 機械学習を構成する要素
## - 機械学習モデルとは
## T- ensorFlowとPythonによるautoencoderの実装
## - 画像分類技術の基礎
## - TensorFlowとPythonによる画像分類の実装
## (- 強化学習の基礎?)
## (- 迷路探索アルゴリズムの実装?)